<a href="https://colab.research.google.com/github/byu-cs-452/byu-cs-452-class-content/blob/main/sparksql/03%20-%20Covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup Spark SQL
# Note if running locally you need the JVM https://www.oracle.com/java/technologies/downloads/
# Also, if running locally you'll need to allow it to talk over the network to your own machine
# Consider running in https://colab.research.google.com/
%pip install pyspark

In [1]:
# Initialize Context - this is where you'd setup information about your Hadoop cluster if you had one!
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Covid").getOrCreate()

sc = spark.sparkContext

sc.setLogLevel("WARN")

In [2]:
# Download 100mb covid county data file
!curl "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv" > ./uscounties.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0 99.9M    0  6890    0     0   3554      0  8:11:26  0:00:01  8:11:25  3557
  5 99.9M    5 5213k    0     0  2336k      0  0:00:43  0:00:02  0:00:41 2337k
 22 99.9M   22 22.0M    0     0  6995k      0  0:00:14  0:00:03  0:00:11 6998k
 43 99.9M   43 43.5M    0     0  10.2M      0  0:00:09  0:00:04  0:00:05 10.2M
 64 99.9M   64 64.4M    0     0  12.3M      0  0:00:08  0:00:05  0:00:03 15.5M
 85 99.9M   85 85.5M    0     0  13.7M      0  0:00:07  0:00:06  0:00:01 19.9M
100 99.9M  100 99.9M    0     0  14.4M      0  0:00:06  0:00:06 --:--:-- 20.1M


In [ ]:
# Write code to define or infer the schema and then read in the dataset
# Read the file into a Spark DataFrame
usCountiesFilePath = "./uscounties.csv"
from pyspark.sql.types import DateType, StringType, StructField, StructType, IntegerType

# the fips column needs to be a string in order to preserve leading zeros
schema = StructType([StructField('date', DateType(), True), StructField('county', StringType(), True), StructField('state', StringType(), True), StructField('fips', StringType(), True), StructField('cases', IntegerType(), True), StructField('deaths', IntegerType(), True)])

df = spark.read.csv(usCountiesFilePath, schema=schema, header=True)

StructType([StructField('date', DateType(), True), StructField('county', StringType(), True), StructField('state', StringType(), True), StructField('fips', StringType(), True), StructField('cases', IntegerType(), True), StructField('deaths', IntegerType(), True)])

In [8]:
# Write code to find the county with the most deaths
# Option 1) SparkSQL API
df.createOrReplaceTempView("covid")  # create table that you can do sql on

print("Max deaths:")
spark.sql(
    """
    select county, state, deaths
    from covid
    order by deaths desc
    limit 1
  """
).show()

Max deaths:
+-------------+--------+------+
|       county|   state|deaths|
+-------------+--------+------+
|New York City|New York| 40267|
+-------------+--------+------+



In [5]:
# Option 2) # DataFrame style
from pyspark.sql.functions import col

print("Max deaths:")
print(
    df.orderBy(col("deaths").desc()).take(  # .where(col("county") == "New York City") \
        1
    )
)

Max deaths:
[Row(date=datetime.date(2022, 5, 13), county='New York City', state='New York', fips=None, cases=2422658, deaths=40267)]


In [5]:
# # Option 3) RDD MapReduce Style without key
rows = df.rdd


def getMax(cumm, other):
    if other["deaths"] is not None and other["deaths"] > cumm["deaths"]:
        return other
    else:
        return cumm


print("Max deaths:")
print(rows.reduce(getMax))

Max deaths:


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 14 in stage 3.0 failed 1 times, most recent failure: Lost task 14.0 in stage 3.0 (TID 32) (10.89.80.224 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\Sarah\Code\CS_452\.venv\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1247, in main
    process()
  File "C:\Users\Sarah\Code\CS_452\.venv\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "C:\Users\Sarah\Code\CS_452\.venv\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Sarah\Code\CS_452\.venv\Lib\site-packages\py

In [7]:
# # Option 4) RDD MapReduce Style with mapped tuples
rows = df.rdd


def getMax(cumm, other):
    if other[0] > cumm[0]:
        return other
    else:
        return cumm


rows = rows.map(lambda r: (r["deaths"] or 0, f"{r['county']},{r['state']}"))
print("Max deaths:")
print(rows.reduce(getMax))

Max deaths:
(40267, 'New York City,New York')


In [48]:
# Write code to find the county with the most deaths
df.createOrReplaceTempView("covid")  # create table that you can do sql on

print("Max deaths:")
spark.sql(
    """
    select county, state, deaths
    from covid
    order by deaths desc
    limit 1
  """
).show()

Max deaths:
+-------------+--------+------+
|       county|   state|deaths|
+-------------+--------+------+
|New York City|New York| 40267|
+-------------+--------+------+



In [ ]:
latest_date = '2022-05-13'

AttributeError: 'str' object has no attribute 'toordinal'

In [9]:
# Write code to find the county with the most cases
print("Max cases:")
spark.sql(
    """
    select county, state, cases
    from covid
    order by cases desc
    limit 1
  """
).show()

Max cases:
+-----------+----------+-------+
|     county|     state|  cases|
+-----------+----------+-------+
|Los Angeles|California|2908425|
+-----------+----------+-------+



In [39]:
# Write code to find the total number of deaths in Utah county
print("Total deaths in Utah County:")
spark.sql(
  f"""
    with latest_fips as (
      select fips, max(date) as latest
      from covid
      group by fips
    )
    select county, state, deaths
    from covid
    where county='Utah' and date='2022-05-13'
  """
).show()

Total deaths in Utah County:
+------+-----+------+
|county|state|deaths|
+------+-----+------+
|  Utah| Utah|   791|
+------+-----+------+



In [40]:
# Write code to find the death rate for each state and sort the states by death rate descending
print("Death rate for each state:")
spark.sql(
    """
    select state, sum(deaths) / sum(cases) as rate
    from covid
    where date='2022-05-13'
    group by state
    order by rate desc
  """
).show()

Death rate for each state:
+-------------+--------------------+
|        state|                rate|
+-------------+--------------------+
| Pennsylvania|0.015722318331513083|
|  Mississippi|0.015541584999631953|
|      Alabama|0.015043956128181742|
|       Nevada|0.014900913477587933|
|      Arizona|0.014884843113359676|
|      Georgia|0.014874971808464166|
|     Michigan|0.014616217125644464|
|   New Jersey|0.014498962846650889|
|   New Mexico|0.014462012745729724|
|     Missouri|0.014159391860103612|
|         Ohio| 0.01415176937498371|
|  Connecticut|0.014002001385574628|
|    Louisiana|0.013898007349590935|
|     Maryland|0.013799618514678251|
|      Indiana|0.013787799751901293|
|     Oklahoma|0.013771585140095712|
|West Virginia|0.013635248690478075|
|     Arkansas| 0.01361763958527935|
|    Tennessee|0.013055661731134637|
|        Texas|0.013021050347158319|
+-------------+--------------------+
only showing top 20 rows



In [ ]:
# Write code to something else interesting with this data – your choice
print("Top days with the highest case increases in Utah County:")
spark.sql(
    """
    select curr.date, curr.cases - prev.cases as case_increase
    from covid curr
    left join covid prev on prev.date = curr.date - 1
    where curr.county='Utah' and prev.county='Utah'
    order by case_increase desc
  """
).show()

Top 10 days with the highest case increases in Utah County:
+----------+-------------+
|      date|case_increase|
+----------+-------------+
|2022-01-18|         8221|
|2022-01-24|         4756|
|2022-01-10|         4463|
|2022-01-19|         2990|
|2022-01-21|         2716|
|2022-01-20|         2625|
|2022-01-03|         2569|
|2022-01-13|         2563|
|2022-01-14|         1984|
|2022-01-31|         1911|
|2021-01-13|         1878|
|2022-01-07|         1848|
|2022-01-11|         1766|
|2022-01-12|         1628|
|2022-01-06|         1620|
|2022-01-25|         1598|
|2022-01-27|         1542|
|2020-11-27|         1482|
|2022-01-26|         1425|
|2020-11-14|         1335|
+----------+-------------+
only showing top 20 rows



In [ ]:
# Extra Credit 1 - Plot your death rate data!
# Extra Credit 2 - Join this with other data or find something intresting in this data and plot it on a map!

In [1]:
# This example uses two-letter state code

import pandas as pd
import plotly.express as px

data = pd.DataFrame({
  'state': ['NY', 'CA', 'TX', 'FL'],
  'values': [10, 20, 15, 25]
})

fig = px.choropleth(
    data,
    locations='state', # Column with state abbreviations
    locationmode='USA-states', # Set location mode to US states
    color='values', # Column to determine color intensity
    scope='usa', # Limit map to the USA
    color_continuous_scale='Viridis', # Choose a color scale
    title='Extra Credit Plot <Insert name here>'
)

fig.show()


In [35]:
# This example uses the FIPS code to map data to a county

import pandas as pd
import plotly.express as px

# Example county-level data (FIPS codes are required for county-level plots)
data = pd.DataFrame({
    'fips': ['36061', '06037', '48201', '12086'],  # Example FIPS codes (NYC, LA, Houston, Miami-Dade)
    'values': [10, 20, 15, 25]
})

# Plot county-level choropleth map
fig = px.choropleth(
    data,
    geojson="https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json",  # GeoJSON for counties
    locations='fips',  # Use county FIPS codes
    color='values',  # Column to determine color intensity
    color_continuous_scale='Viridis',
    scope='usa',
    title='County-Level Extra Credit Plot'
)

fig.show()


In [53]:
county_death_rate = spark.sql(
  """
    select fips, deaths / cases as rate
    from covid
    where fips is not Null and date='2022-05-13'
  """
)

In [54]:
import plotly.express as px

deathrate_pd = county_death_rate.toPandas()

fig = px.choropleth(
    deathrate_pd,
    geojson="https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json",  # GeoJSON for counties
    locations='fips',  # Use county FIPS codes
    color='rate',  # Column to determine color intensity
    color_continuous_scale='Viridis',
    scope='usa',
    title='County-Level Death Rate'
)

fig.show()
